## 当代人工智能 Project1

In [1]:
import numpy as np
import pandas as pd
import random

先在Colab中安装Transformers库

In [2]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 636 kB 38.1 MB/s 
     |████████████████████████████████| 895 kB 40.9 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 3.3 MB 30.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import pipeline

In [4]:
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import Trainer, TrainingArguments
from transformers import DistilBertForSequenceClassification

In [5]:
import matplotlib.pyplot as plt

In [6]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [7]:
import torch

In [8]:
from sklearn.metrics import accuracy_score

In [9]:
distill_bert_model = "distilbert-base-uncased"

In [10]:
distill_bert_tokenizer = DistilBertTokenizer.from_pretrained(distill_bert_model, do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

此函数实现从20newsgroup中读取所有数据，并且完成数据集分割

In [11]:
def newsgroup_read(test_size=0.2):
  dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))
  news_text = dataset.data
  news_label = dataset.target
  return train_test_split(news_text, news_label, test_size=test_size), dataset.target_names

次函数实现从20newsgroup中读取'alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space'，'comp.os.ms-windows.misc'五个主题的数据，并且完成数据集分割

In [12]:
def newsgroup_read_selected(test_size=0.2):
  categories = ['alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space', 'comp.os.ms-windows.misc']
  dataset = fetch_20newsgroups(subset='all', shuffle=True, remove=("headers", "footers", "quotes"), categories = categories)
  news_text = dataset.data
  news_label = dataset.target
  return train_test_split(news_text, news_label, test_size=test_size), dataset.target_names

In [ ]:
(train_news_texts, valid_news_texts, train_news_labels, valid_news_labels), target_names = newsgroup_read()

In [13]:
(selected_train_news_texts, selected_valid_news_texts, selected_train_news_labels, selected_valid_news_labels), selected_target_names = newsgroup_read_selected()

In [14]:
max_length = 256
# max_length = 512
# 512在大部分情况下Colab会报出内存不足的错误

In [15]:
selected_train_encodings = distill_bert_tokenizer(selected_train_news_texts, truncation=True, padding=True, max_length=max_length)
selected_valid_encodings = distill_bert_tokenizer(selected_valid_news_texts, truncation=True, padding=True, max_length=max_length)

In [16]:
class GetNewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [17]:
selected_train_dataset = GetNewsGroupsDataset(selected_train_encodings, selected_train_news_labels)
selected_valid_dataset = GetNewsGroupsDataset(selected_valid_encodings, selected_valid_news_labels)

In [19]:
selected_classification_model = DistilBertForSequenceClassification.from_pretrained(distill_bert_model,num_labels=len(selected_target_names)) # to("cuda")

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

在这个BERT实验中，我们先使用原始的20分类任务，也考虑使用5分类任务

In [20]:
training_args = TrainingArguments(
    output_dir='./selected_distill_bert_model_results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=12,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    logging_steps=250,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [21]:
def compute_model_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  model_acc = accuracy_score(labels, preds)
  return {
      'Distill_Bert_accuracy': model_acc,
  }

trainer = Trainer(
    model=selected_classification_model,                     
    args=training_args,                 
    train_dataset=selected_train_dataset,       
    eval_dataset=selected_valid_dataset,          
    compute_metrics=compute_model_metrics,   
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 3497
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4380


Step,Training Loss,Validation Loss,Distill Bert Accuracy
250,1.019000,0.643939,0.728000
500,0.596600,0.577520,0.792000
750,0.451200,0.725953,0.772571
1000,0.412600,0.680923,0.793143
1250,0.253400,0.813667,0.797714
1500,0.207700,0.962986,0.787429
1750,0.168000,1.038570,0.797714
2000,0.104600,1.034430,0.796571
2250,0.098700,1.015270,0.805714
2500,0.081800,1.139912,0.789714


***** Running Evaluation *****
  Num examples = 875
  Batch size = 12
Saving model checkpoint to ./selected_distill_bert_model_results/checkpoint-250
Configuration saved in ./selected_distill_bert_model_results/checkpoint-250/config.json
Model weights saved in ./selected_distill_bert_model_results/checkpoint-250/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 875
  Batch size = 12
Saving model checkpoint to ./selected_distill_bert_model_results/checkpoint-500
Configuration saved in ./selected_distill_bert_model_results/checkpoint-500/config.json
Model weights saved in ./selected_distill_bert_model_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 875
  Batch size = 12
Saving model checkpoint to ./selected_distill_bert_model_results/checkpoint-750
Configuration saved in ./selected_distill_bert_model_results/checkpoint-750/config.json
Model weights saved in ./selected_distill_bert_model_results/checkpoint-750/pytorch_model.bin
****

In [ ]:
class GetNewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = GetNewsGroupsDataset(train_encodings, train_news_labels)
valid_dataset = GetNewsGroupsDataset(valid_encodings, valid_news_labels)

In [ ]:
# model = DistilBertForSequenceClassification.from_pretrained(distill_bert_model,num_labels=len(target_names)).to("cuda")
model = DistilBertForSequenceClassification.from_pretrained(distill_bert_model,num_labels=len(target_names))

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "L

In [ ]:
training_args = TrainingArguments(
    output_dir='./distill_bert_model_results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=12,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    logging_steps=250,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

using `logging_steps` to initialize `eval_steps` to 250
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def compute_model_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  model_acc = accuracy_score(labels, preds)
  return {
      'Distill_Bert_accuracy': model_acc,
  }

trainer = Trainer(
    model=model,                     
    args=training_args,                 
    train_dataset=train_dataset,       
    eval_dataset=valid_dataset,          
    compute_metrics=compute_model_metrics,   
)

开始Fine Tune Distill BERT模型

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 15076
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18850


Step,Training Loss,Validation Loss,Distill Bert Accuracy
250,2.384200,1.590589,0.518568
500,1.351900,1.227081,0.632891
750,1.182500,1.180167,0.644562
1000,1.118700,1.112769,0.661804
1250,1.083200,1.067094,0.677719
1500,1.034900,1.008571,0.687268
1750,0.978700,1.090473,0.664721
2000,0.843000,1.034891,0.700265


***** Running Evaluation *****
  Num examples = 3770
  Batch size = 12
Saving model checkpoint to ./distill_bert_model_results/checkpoint-250
Configuration saved in ./distill_bert_model_results/checkpoint-250/config.json
Model weights saved in ./distill_bert_model_results/checkpoint-250/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3770
  Batch size = 12
Saving model checkpoint to ./distill_bert_model_results/checkpoint-500
Configuration saved in ./distill_bert_model_results/checkpoint-500/config.json
Model weights saved in ./distill_bert_model_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3770
  Batch size = 12
Saving model checkpoint to ./distill_bert_model_results/checkpoint-750
Configuration saved in ./distill_bert_model_results/checkpoint-750/config.json
Model weights saved in ./distill_bert_model_results/checkpoint-750/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3770
  Batch size = 12
Saving mode